In [1]:
from pulp import *
import os
import numpy as np
import pandas as pd
import gurobipy as grb

def read_matrix(path_file):
    satisfactions = []
    resources = []
    capacities = []
    
    with open(path_file, 'r') as f:
        m, n= f.readline().split(' ')[:2]
        m, n = np.int32(m), np.int32(n)
        
        lines = f.readlines()
        lines = [line.rstrip().split(' ') for line in lines]

        for i in range(0, m):
            for j in range(n):
                satisfactions.append(np.int32(lines[i][j]))

        for i in range(m, m + m):
            for j in range(n):
                resources.append(np.int32(lines[i][j]))
        
        for j in range(m):
            capacities.append(np.int32(lines[2*m][j]))
            
    satisfactions = np.array(satisfactions)
    resources = np.array(resources)
    capacities = np.array(capacities)
    
    return m, n, satisfactions, resources, capacities

In [2]:
# References: 
# How to keep log files: https://coin-or.github.io/pulp/guides/how_to_configure_solvers.html

In [3]:
import os
import json

def solve(inputs):
    solver = getSolver('GUROBI', TimeLimit=60, Presolve=0, VarBranch=0,
                          Cuts=0, BranchDir=-1)
    for inputPath in inputs:
        prob = LpProblem("desig_gen_problem", LpMaximize)

        workers, tasks, satisfaction, resource, capacity = read_matrix(os.path.join("./input", inputPath))

        N = range(workers*tasks)

        x = LpVariable.dicts(
            "x", N, lowBound=0, upBound=1, cat=LpInteger
        )

        prob += lpSum([satisfaction[i] * x[i]  for i in N]), 'objective F'

        #   every job can only be picked by one worker
        for i in range(tasks):
            prob += lpSum([x[i + j*tasks] for j in range(workers)]) == 1

        #   every worker has to take only the jobs that they can
        for i in range(workers):
            prob += lpSum([x[i*tasks + j] * resource[i*tasks + j] for j in range(tasks)]) <= capacity[i]
        
        prob.solve(GUROBI())

        sol = json.loads(prob.solverModel.getJSONSolution())
        
        print(json.dumps(sol["SolutionInfo"], indent=4))

In [4]:
solve(os.listdir("./input/"))

Set parameter Username
Academic license - for non-commercial use only - expires 2022-01-30
Set parameter TimeLimit to value 60
Set parameter Presolve to value 0
Set parameter VarBranch to value 0
Set parameter Cuts to value 0
Set parameter BranchDir to value -1
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1620 rows, 32000 columns and 64000 nonzeros
Model fingerprint: 0x439d68df
Variable types: 0 continuous, 32000 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+03]
Variable types: 0 continuous, 32000 integer (32000 binary)
Found heuristic solution: objective -169195.0000

Root relaxation: objective -9.782135e+04, 4409 iterations, 0.12 seconds (0.13 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  

  1410  1711 -54551.000  243  115 -54720.000 -54551.000  0.31%   5.9   55s
  2277  2388 -54551.000  384  116 -54720.000 -54551.000  0.31%   5.7   60s

Explored 2387 nodes (22862 simplex iterations) in 60.06 seconds (36.97 work units)
Thread count was 12 (of 12 available processors)

Solution count 10: -54720 -54723 -54724 ... -54860

Time limit reached
Best objective -5.472000000000e+04, best bound -5.455100000000e+04, gap 0.3088%
Gurobi status= 9
{
    "Status": 9,
    "Runtime": 60.07198524475098,
    "Work": 36.96956569209642,
    "ObjVal": -54720,
    "ObjBound": -54551,
    "ObjBoundC": -54551,
    "MIPGap": 0.0030884502923976607,
    "IntVio": 0,
    "BoundVio": 0,
    "ConstrVio": 0,
    "IterCount": 22862,
    "BarIterCount": 0,
    "NodeCount": 2387,
    "SolCount": 10,
    "PoolObjBound": -54551,
    "PoolObjVal": [
        -54720,
        -54723,
        -54724,
        -54725,
        -54728,
        -54806,
        -54825,
        -54841,
        -54844,
        -54860
   


Solution count 9: -176886 -176921 -176932 ... -528558

Time limit reached
Best objective -1.768860000000e+05, best bound -1.767860000000e+05, gap 0.0565%
Gurobi status= 9
{
    "Status": 9,
    "Runtime": 60.17707824707031,
    "Work": 43.84001454762865,
    "ObjVal": -176886,
    "ObjBound": -176786,
    "ObjBoundC": -176785.22016753818,
    "MIPGap": 0.0005653358660380131,
    "IntVio": 0,
    "BoundVio": 0,
    "ConstrVio": 0,
    "IterCount": 19512,
    "BarIterCount": 0,
    "NodeCount": 37,
    "SolCount": 9,
    "PoolObjBound": -176786,
    "PoolObjVal": [
        -176886,
        -176921,
        -176932,
        -176934,
        -176955,
        -177434,
        -177695,
        -520440,
        -528558
    ]
}
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 5 rows, 6 columns and 12 nonzeros
Model fingerprint: 0x0309ae54
Variable types: 0 continuous, 6 integer (0 binary)